# Your First Mechanistic Interpretability Analysis

A complete walkthrough from question to understanding.

**Companion notebook for [First Principles of Mechanistic Interpretability](https://ttsugriy.github.io/mechinterp-first-principles/first-analysis.html)**

---

## What You'll Do

1. Define a simple behavior to analyze
2. Verify the model exhibits the behavior
3. Use attribution to find relevant components
4. Validate with patching
5. Interpret what you found

**Prerequisites:** Make sure you have a GPU runtime (Runtime → Change runtime type → T4 GPU)

## Step 0: Setup

In [ ]:
# Install libraries
!pip install -q transformer-lens einops jaxtyping circuitsvis plotly

import torch
import transformer_lens as tl
from transformer_lens import utils
import plotly.express as px
import pandas as pd

# Load GPT-2 Small
model = tl.HookedTransformer.from_pretrained("gpt2-small")
print(f"Loaded {model.cfg.model_name}: {model.cfg.n_layers} layers, {model.cfg.n_heads} heads")

---
## The Behavior: Sentiment-Influenced Completion

We'll analyze a simple but interesting behavior:

> When given "I love this movie because it is", GPT-2 predicts positive words like "great" or "good".
> When given "I hate this movie because it is", GPT-2 predicts negative words like "bad" or "terrible".

**The question**: How does the model know to predict positive vs negative words? Which components read the sentiment word ("love" vs "hate") and influence the prediction?

## Step 1: Verify the Behavior Exists

**Never skip this step.** Before analyzing a behavior, confirm the model actually exhibits it.

In [ ]:
def get_top_predictions(model, prompt: str, k: int = 5) -> list[tuple[str, float, float]]:
    """Get the model's top-k predictions for the next token."""
    tokens = model.to_tokens(prompt)
    logits = model(tokens)
    top_logits, top_tokens = logits[0, -1].topk(k)
    probs = torch.softmax(logits[0, -1], dim=-1)

    return [
        (model.tokenizer.decode(token), probs[token].item(), logit.item())
        for logit, token in zip(top_logits, top_tokens)
    ]

# Test positive sentiment
print("POSITIVE: 'I love this movie because it is'")
for word, prob, logit in get_top_predictions(model, "I love this movie because it is"):
    print(f"  {word:15} prob={prob:.3f}  logit={logit:.2f}")

print("\nNEGATIVE: 'I hate this movie because it is'")
for word, prob, logit in get_top_predictions(model, "I hate this movie because it is"):
    print(f"  {word:15} prob={prob:.3f}  logit={logit:.2f}")

In [ ]:
# Check specific sentiment words
def get_token_logit(model, prompt: str, target_word: str) -> float:
    """Get the logit for a specific target word."""
    tokens = model.to_tokens(prompt)
    logits = model(tokens)
    target_token = model.to_single_token(target_word)
    return logits[0, -1, target_token].item()

# Compare logits for "great" and "bad"
positive_prompt = "I love this movie because it is"
negative_prompt = "I hate this movie because it is"

print("Logit for 'great':")
print(f"  After 'love': {get_token_logit(model, positive_prompt, ' great'):.2f}")
print(f"  After 'hate': {get_token_logit(model, negative_prompt, ' great'):.2f}")

print("\nLogit for 'bad':")
print(f"  After 'love': {get_token_logit(model, positive_prompt, ' bad'):.2f}")
print(f"  After 'hate': {get_token_logit(model, negative_prompt, ' bad'):.2f}")

✅ **Confirmed!** The model shifts predictions based on sentiment:
- " great" gets higher logit after "love"
- " bad" gets higher logit after "hate"

---
## Step 2: Logit Attribution

Now let's find *which components* are responsible for this difference.

**The idea**: Decompose the final logit into per-component contributions. Which attention heads and MLPs push toward " great" in the positive case?

In [ ]:
def logit_attribution(model, prompt: str, target_token: str) -> tuple[dict[str, float], float]:
    """
    Compute each component's contribution to the target token's logit.
    Returns a dict of component -> contribution, and the total logit.
    """
    target_id = model.to_single_token(target_token)
    tokens = model.to_tokens(prompt)

    # run_with_cache stores all intermediate activations for analysis
    logits, cache = model.run_with_cache(tokens)

    # The unembedding column for our target - this is the "great direction"
    # in residual stream space that, when projected onto, gives the logit
    target_dir = model.W_U[:, target_id]

    # Measure how much each component points toward our target
    # Positive = helps predict it, negative = suppresses it
    contributions = {
        "embed": (cache["embed"][0, -1] @ target_dir).item(),
        "pos_embed": (cache["pos_embed"][0, -1] @ target_dir).item(),
    }

    for layer in range(model.cfg.n_layers):
        contributions[f"L{layer}_attn"] = (cache["attn_out", layer][0, -1] @ target_dir).item()
        contributions[f"L{layer}_mlp"] = (cache["mlp_out", layer][0, -1] @ target_dir).item()

    return contributions, logits[0, -1, target_id].item()

In [ ]:
# Get attribution for " great" in positive case
pos_attrib, pos_logit = logit_attribution(model, positive_prompt, " great")
neg_attrib, neg_logit = logit_attribution(model, negative_prompt, " great")

print(f"Total logit for ' great': positive={pos_logit:.2f}, negative={neg_logit:.2f}")
print(f"Difference: {pos_logit - neg_logit:.2f}")

In [ ]:
# Compare attributions - which components differ most?
diff_attrib = {k: pos_attrib[k] - neg_attrib[k] for k in pos_attrib}

# Sort by absolute difference
sorted_diff = sorted(diff_attrib.items(), key=lambda x: abs(x[1]), reverse=True)

print("\nComponents with largest attribution difference (positive - negative):")
print("(Positive values → component pushes more toward 'great' in positive case)")
print("-" * 60)
for component, diff in sorted_diff[:10]:
    print(f"{component:15} diff={diff:+.3f}  (pos={pos_attrib[component]:.3f}, neg={neg_attrib[component]:.3f})")

💡 **Key Insight**: The components with the *largest difference* between positive and negative cases are the ones that "read" the sentiment and adjust the prediction accordingly.

---
## Step 3: Visualize the Attribution

In [ ]:
# Create a visualization
data = []
for layer in range(model.cfg.n_layers):
    data.append({
        "layer": layer,
        "component": "attention",
        "positive": pos_attrib[f"L{layer}_attn"],
        "negative": neg_attrib[f"L{layer}_attn"],
        "diff": pos_attrib[f"L{layer}_attn"] - neg_attrib[f"L{layer}_attn"]
    })
    data.append({
        "layer": layer,
        "component": "mlp",
        "positive": pos_attrib[f"L{layer}_mlp"],
        "negative": neg_attrib[f"L{layer}_mlp"],
        "diff": pos_attrib[f"L{layer}_mlp"] - neg_attrib[f"L{layer}_mlp"]
    })

df = pd.DataFrame(data)

# Plot the difference
fig = px.bar(df, x="layer", y="diff", color="component",
             barmode="group",
             title="Attribution Difference: Positive vs Negative Sentiment",
             labels={"diff": "Contribution difference to 'great'", "layer": "Layer"})
fig.show()

**What to look for**: Layers where the bars are tall (large difference) are where the model processes sentiment.

---
## Step 4: Look at Specific Heads

The layer-level view is coarse. Let's look at individual attention heads.

In [ ]:
def head_attribution(model, prompt, target_token):
    """Get per-head attribution."""
    target_id = model.to_single_token(target_token)
    tokens = model.to_tokens(prompt)
    logits, cache = model.run_with_cache(tokens)

    target_dir = model.W_U[:, target_id]

    contributions = {}
    for layer in range(model.cfg.n_layers):
        # Get per-head outputs (after W_O projection)
        # Shape: [batch, seq, n_heads, d_model]
        head_results = cache["result", layer][0, -1]  # [n_heads, d_model]

        for head in range(model.cfg.n_heads):
            head_out = head_results[head]  # [d_model]
            contributions[f"L{layer}H{head}"] = (head_out @ target_dir).item()

    return contributions

pos_heads = head_attribution(model, positive_prompt, " great")
neg_heads = head_attribution(model, negative_prompt, " great")

# Find heads with biggest difference
head_diff = {k: pos_heads[k] - neg_heads[k] for k in pos_heads}
sorted_heads = sorted(head_diff.items(), key=lambda x: abs(x[1]), reverse=True)

print("Top 10 heads by attribution difference:")
print("-" * 50)
for head, diff in sorted_heads[:10]:
    print(f"{head}: diff={diff:+.3f}")

📝 **Write down the top heads.** These are our hypotheses for "sentiment-reading heads." We'll validate them next.

---
## Step 5: Validate with Patching

Attribution is correlational. Now we test causation: if we patch the sentiment word's representation from the negative case into the positive case, does the prediction change?

In [ ]:
def patch_position(model, clean_prompt, corrupted_prompt, position, layer):
    """
    Patch the residual stream at a specific position and layer
    from corrupted into clean. This is our causal intervention—we're
    changing what the model sees mid-computation.
    """
    corrupted_tokens = model.to_tokens(corrupted_prompt)
    _, corrupted_cache = model.run_with_cache(corrupted_tokens)

    # Get the residual stream after this layer processes
    # This contains all information the model has built up to this point
    corrupted_resid = corrupted_cache["resid_post", layer]

    # A hook intercepts and optionally modifies activations during forward pass
    # Here we surgically replace just one position's representation
    def patch_hook(resid, hook):
        resid[:, position, :] = corrupted_resid[:, position, :]
        return resid

    clean_tokens = model.to_tokens(clean_prompt)

    # run_with_hooks executes forward pass but calls our hook at the specified location
    patched_logits = model.run_with_hooks(
        clean_tokens,
        fwd_hooks=[(f"blocks.{layer}.hook_resid_post", patch_hook)]
    )

    return patched_logits

In [ ]:
# Find position of "love" / "hate"
pos_tokens = model.to_str_tokens(positive_prompt)
print(f"Tokens: {pos_tokens}")
sentiment_pos = 1  # Usually "love" is at position 1 (after "I")

# Patch at different layers
print("\nPatching 'love' → 'hate' at sentiment position:")
print("-" * 50)

clean_logit = get_token_logit(model, positive_prompt, " great")
print(f"Original (love → great): {clean_logit:.2f}")

for layer in range(model.cfg.n_layers):
    patched = patch_position(model, positive_prompt, negative_prompt, sentiment_pos, layer)
    target_id = model.to_single_token(" great")
    patched_logit = patched[0, -1, target_id].item()
    change = patched_logit - clean_logit
    print(f"Layer {layer:2d}: {patched_logit:.2f} (change: {change:+.2f})")

**What to look for**: Layers where patching causes a big drop in "great" logit are where the sentiment information is being used.

---
## Step 6: Examine Attention Patterns

Let's see what the important heads are attending to.

In [ ]:
import circuitsvis as cv
from IPython.display import display

# Get attention patterns
tokens = model.to_tokens(positive_prompt)
_, cache = model.run_with_cache(tokens)

str_tokens = model.to_str_tokens(positive_prompt)

# Pick a head that showed up in attribution (replace with your findings!)
important_layer = 8  # Example - use your results
important_head = 6   # Example - use your results

# Get attention pattern for this head
pattern = cache["pattern", important_layer][0, important_head]  # [seq, seq]

print(f"Attention pattern for L{important_layer}H{important_head}")
print(f"Tokens: {str_tokens}")

# Visualize
display(cv.attention.attention_patterns(
    attention=cache["pattern", important_layer][0:1],  # [1, n_heads, seq, seq]
    tokens=str_tokens,
    attention_head_names=[f"H{h}" for h in range(model.cfg.n_heads)]
))

**What to look for**: Does the final position attend to the sentiment word ("love")?

---
## Step 7: Interpret Your Findings

Based on your analysis, answer these questions:

1. **Which layers process sentiment?** (Where did patching have the biggest effect?)
2. **Which heads are involved?** (Which had the biggest attribution difference?)
3. **What are they attending to?** (Do they attend to the sentiment word?)
4. **How does the information flow?** (Early layers read sentiment → later layers adjust prediction?)

### ✍️ Write Your Interpretation

Before continuing, write 2-3 sentences describing what you found:

*Example: "Layers 7-9 show the biggest patching effect, suggesting sentiment is processed in mid-to-late layers. Head 8.6 has the largest attribution difference and attends strongly to the sentiment word at the final position. This suggests a 'sentiment → adjective' circuit where..."*

**Your interpretation:**

(Double-click to edit this cell and write here)

---
## Step 8: Sanity Checks

Before claiming you understand the circuit, verify it generalizes.

In [ ]:
# 1. Does it generalize to other examples?
test_cases = [
    ("I really love this book because it is", "positive"),
    ("I absolutely hate this song because it is", "negative"),
    ("This restaurant is great because the food is", "positive"),
    ("This restaurant is terrible because the food is", "negative"),
]

print("Generalization test:")
for prompt, expected in test_cases:
    great_logit = get_token_logit(model, prompt, " great")
    bad_logit = get_token_logit(model, prompt, " bad")
    predicted = "positive" if great_logit > bad_logit else "negative"
    match = "✓" if predicted == expected else "✗"
    print(f"{match} {expected:8} | great={great_logit:.1f}, bad={bad_logit:.1f} | {prompt[:40]}...")

# 2. What happens without sentiment words?
neutral = "I watched this movie because it is"
print(f"\nNeutral case: great={get_token_logit(model, neutral, ' great'):.2f}, bad={get_token_logit(model, neutral, ' bad'):.2f}")

---
## 🎉 What You've Learned

Congratulations! You've completed a full interpretability analysis. You now know how to:

1. ✅ Define and verify a behavior
2. ✅ Use logit attribution to find candidate components
3. ✅ Validate with activation patching
4. ✅ Examine attention patterns
5. ✅ Interpret findings and check generalization

**The workflow you used**:
```
Observe behavior → Attribute → Patch → Interpret → Verify
```

This is the core loop of mechanistic interpretability research.

---
## 🚀 Try It Yourself

Now extend your analysis! Pick one of these variations:

In [ ]:
# TRY IT: Different sentiment words
# Replace "love"/"hate" with "adore"/"despise" - does the same circuit work?

# YOUR CODE HERE


In [ ]:
# TRY IT: Different behavior entirely
# Analyze: "The capital of France is" → "Paris"
# Which components contribute to the factual recall?

# YOUR CODE HERE


In [ ]:
# TRY IT: Break the behavior
# Find an input where the model gets sentiment "wrong"
# What's different about the attribution?

# YOUR CODE HERE


---
## Next Steps

- **[Running Example](https://ttsugriy.github.io/mechinterp-first-principles/running-example.html)** — See all techniques applied to "Eiffel Tower → Paris"
- **[Exercises](https://ttsugriy.github.io/mechinterp-first-principles/exercises.html)** — Practice problems for each technique
- **[Chapter 10: Attribution](https://ttsugriy.github.io/mechinterp-first-principles/chapters/10-attribution.html)** — Deep dive into attribution methods